# Using MaxEnt to compute fluxes in a Toy Network

The Toy Network depicted in Figure 1.2.A can be solved using the [`CasADi`](https://web.casadi.org/) library. To install `CasADi` open a terminal and execute:

`pip install casadi`

Make sure to install version `3.6.0` (as presented above) as later versions conflict with the non-linear programming solver `ipopt`, which `CasADi` uses to maximize the entropy. After installing `CasADi`, come back to this jupyter notebook and load `CasADi` functions.

In [1]:
from casadi import *

Define the variables for each reaction flux:

In [2]:
v1, v2, v3, v4 = MX.sym('v1'), MX.sym('v2'), MX.sym('v3'), MX.sym('v4')

Calculate total flux.

In [3]:
V = v1 + v2 + v3 + v4

as this will came handy to define probabilites as $p_i=v_i/V$. For the entropy $H=-\sum_i p_i\log p_i$, we will use a helper function computing each $p_i \log p_i$ term.

In [4]:
def entropy_term(vi, V):
    # Returns the pilogpi term, where pi=vi/V
    # if pi=0, pilogpi=0
    pi=vi/V
    return if_else(pi == 0, 0, pi * log(pi))

Thus, we write $H$ as:

In [5]:
H = -sum(entropy_term(vi, V) for vi in [v1, v2, v3, v4])

Now, we need to add the mass balance constraints of metabolites `A` and `B`:

In [6]:
MB_A = v1 - v2 - v3
MB_B = v2 + v3 - v4

At this stage we have the objective function, the decision variales, and mass balances. Wrapping everything into a dictionary, `nlp`, storing our non-linear programming problem.

In [7]:
nlp = {
    'x': vertcat(v1, v2, v3, v4),  # Decision variables
    'f': -H,                       # Objective: Maximize entropy
    'g': vertcat(MB_A, MB_B)             # Constraints: Mass balances
}

`CasADi` uses several solvers depending on the type of optimization structure. For a non-linear optimization one, we use the `ipopt` solver.

In [8]:
solver = nlpsol('solver', 'ipopt', nlp)

Finally, we run the optimization by specifying an initial guess on the desicion variables (`x0`), the variables upper (`ubx`) and lower (`lbx`) bounds, and the constraints upper (`ubg`) and lower (`lbg`) bounds.

In [9]:
solution = solver(x0=[10, 10, 0, 10],
                  ubg=0, lbg=0, # Steady-state mass balances
                  lbx=[10, 0, 0, 0], # Fixing only v1 as 10
                  ubx=[10, 1000, 1000, 1000])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        5
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality c

The message `EXIT: Optimal Solution Found` signals that `CasADi` has found the distribution of fluxes maximizing `H`. Explore the results stored in the `solution` dictionary.

In [10]:
print(solution)

{'f': DM(-1.32966), 'g': DM([0, 0]), 'lam_g': DM([-0.00770164, 0.00770164]), 'lam_p': DM(0x0), 'lam_x': DM([7.46722e-10, -4.98791e-10, -4.98541e-10, -2.48059e-10]), 'x': DM([10, 5, 5, 10])}


In particular, the `v1`, `v2`, `v3`, and `v4` values are stored in the key `x`.

In [11]:
print(solution['x'])

[10, 5, 5, 10]
